<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/FinRL_PortfolioAllocation_NeurIPS_2020.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Portfolio Allocation

Tutorials to use OpenAI DRL to perform portfolio allocation in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog is based on our paper: FinRL: A Deep Reinforcement Learning Library for Automated Stock Trading in Quantitative Finance, presented at NeurIPS 2020: Deep RL Workshop.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-portfolio-allocation-9b417660c7cd
* Please report any issues to our Github: https://github.com/AI4Finance-Foundation/FinRL/issues
* **Pytorch Version**



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for portfolio alloacation. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A represents the weight of a stock in the porfolio: a ∈ (-1,1). Assume our stock pool includes N stocks, we can use a list [a<sub>1</sub>, a<sub>2</sub>, ... , a<sub>N</sub>] to determine the weight for each stock in the porfotlio, where a<sub>i</sub> ∈ (-1,1), a<sub>1</sub>+ a<sub>2</sub>+...+a<sub>N</sub>=1. For example, "The weight of AAPL in the portfolio is 10%." is [0.1 , ...].

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [32]:
## install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab

!pip install 'shimmy>=2.0'
!pip install pandas_market_calendars
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

zsh:1: command not found: apt-get
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-asm6e6mv
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-asm6e6mv
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit d25d902a6de54931a329adc38a2663e8f576adc4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-install-vaum9bqz/elegantrl_4bb046eefaf740d28d9360a3f574fc08
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-install-vaum9bqz/elegantrl_4


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them.
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

from finrl import config
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
import sys
sys.path.append("../FinRL-Library")

<a id='1.4'></a>
## 2.4. Create Folders

In [2]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).


In [3]:
TRADE_START_DATE = '2023-04-04'
TRADE_END_DATE = '2025-04-07'

In [4]:
df = pd.read_csv('data.csv')

In [5]:
print(df.isnull().sum())

date      0
close     0
high      0
low       0
open      0
volume    0
tic       0
day       0
dtype: int64


In [6]:
df.shape

(24243, 8)

In [7]:
day_values_per_tic = df.groupby('tic')['day'].apply(lambda x: sorted(x.unique())).reset_index()
day_values_per_tic.columns = ['tic', 'unique_days']

# Display
print(day_values_per_tic)

      tic            unique_days
0     agg        [0, 1, 2, 3, 4]
1     bil        [0, 1, 2, 3, 4]
2  btcusd  [0, 1, 2, 3, 4, 5, 6]
3     gld        [0, 1, 2, 3, 4]
4     spy        [0, 1, 2, 3, 4]
5      vb        [0, 1, 2, 3, 4]
6     vnq        [0, 1, 2, 3, 4]
7      vo        [0, 1, 2, 3, 4]
8     vwo        [0, 1, 2, 3, 4]


In [8]:
# Match 5-day and 7-day tickers using apply
tics_5day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(5)))]['tic']
tics_7day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(7)))]['tic']

# Filter the original df
df_5day_full = df[df['tic'].isin(tics_5day)]
df_7day_full = df[df['tic'].isin(tics_7day)]

# Results
print("5-day tickers:", tics_5day.tolist())
print("7-day tickers:", tics_7day.tolist())
print("5-day df shape:", df_5day_full.shape)
print("7-day df shape:", df_7day_full.shape)

5-day tickers: ['agg', 'bil', 'gld', 'spy', 'vb', 'vnq', 'vo', 'vwo']
7-day tickers: ['btcusd']
5-day df shape: (20520, 8)
7-day df shape: (3723, 8)


In [9]:
df.shape

(24243, 8)

In [10]:
df.head()

,date,close,high,low,open,volume,tic,day
0,2015-02-01,228.99,233.790,210.00,218.67,7220.0,btcusd,6
1,2015-02-02,112.20,112.230,112.00,112.06,2792120.0,agg,0
2,2015-02-02,91.46,91.480,91.46,91.48,3557487.0,bil,0
3,2015-02-02,237.83,240.100,220.89,228.39,7421.0,btcusd,0
4,2015-02-02,122.42,123.155,121.82,121.84,8885189.0,gld,0


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [11]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df_5day_full = fe.preprocess_data(df_5day_full)
df_7day_full = fe.preprocess_data(df_7day_full)

Successfully added technical indicators
Successfully added technical indicators


In [12]:
df_5day_full.shape

(20520, 16)

In [13]:
df_7day_full.shape

(3723, 16)

In [14]:
df = pd.concat([df_5day_full, df_7day_full], ignore_index=False)
df.index = range(len(df))  # Assign new sequential index


In [15]:
df.shape

(24243, 16)

In [16]:
df.head(50)

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2015-02-02,112.20,112.2300,112.0000,112.06,2792120.0,agg,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.000000,112.200000,112.200000
1,2015-02-02,91.46,91.4800,91.4600,91.48,3557487.0,bil,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.000000,91.460000,91.460000
2,2015-02-02,122.42,123.1550,121.8200,121.84,8885189.0,gld,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.000000,122.420000,122.420000
3,2015-02-02,201.92,202.0300,197.8600,200.05,163106969.0,spy,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.000000,201.920000,201.920000
4,2015-02-02,115.12,115.2100,112.8500,114.38,549866.0,vb,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.000000,115.120000,115.120000
5,2015-02-02,86.32,86.6900,84.6851,86.53,7523640.0,vnq,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.000000,86.320000,86.320000
6,2015-02-02,122.30,122.3899,119.9400,121.66,823489.0,vo,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.000000,122.300000,122.300000
7,2015-02-02,40.65,40.7600,40.2800,40.40,15944366.0,vwo,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.000000,40.650000,40.650000
8,2015-02-03,111.77,112.1000,111.7600,112.10,1815375.0,agg,1,-0.009647,112.593112,111.376888,0.000000,-66.666667,100.000000,111.985000,111.985000
9,2015-02-03,91.48,91.4800,91.4600,91.46,266084.0,bil,1,0.000449,91.498284,91.441716,100.000000,66.666667,100.000000,91.470000,91.470000


In [17]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Remove rows where the date appears only once
df = df[df.groupby('date')['date'].transform('count') > 1]

# Sort by date
df = df.sort_values('date').reset_index(drop=True)


In [18]:
df.shape

(23085, 16)

In [19]:
df.head(20)

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2015-02-02,112.20,112.2300,112.0000,112.06,2792120.0,agg,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.0,112.200000,112.200000
1,2015-02-02,40.65,40.7600,40.2800,40.40,15944366.0,vwo,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.0,40.650000,40.650000
2,2015-02-02,122.30,122.3899,119.9400,121.66,823489.0,vo,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.0,122.300000,122.300000
3,2015-02-02,86.32,86.6900,84.6851,86.53,7523640.0,vnq,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.0,86.320000,86.320000
4,2015-02-02,237.83,240.1000,220.8900,228.39,7421.0,btcusd,0,0.198333,245.911648,220.908352,100.000000,66.666667,100.0,233.410000,233.410000
5,2015-02-02,201.92,202.0300,197.8600,200.05,163106969.0,spy,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.0,201.920000,201.920000
6,2015-02-02,122.42,123.1550,121.8200,121.84,8885189.0,gld,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.0,122.420000,122.420000
7,2015-02-02,91.46,91.4800,91.4600,91.48,3557487.0,bil,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.0,91.460000,91.460000
8,2015-02-02,115.12,115.2100,112.8500,114.38,549866.0,vb,0,0.000000,112.593112,111.376888,0.000000,-66.666667,100.0,115.120000,115.120000
9,2015-02-03,111.77,112.1000,111.7600,112.10,1815375.0,agg,1,-0.009647,112.593112,111.376888,0.000000,-66.666667,100.0,111.985000,111.985000


## Add covariance matrix as states

In [20]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values
  cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)


In [21]:
df.shape

(20817, 18)

In [22]:
df.head(10)

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2016-02-02,109.32,109.380,109.1301,109.23,4835115.0,agg,1,0.207871,109.414436,108.155564,56.401620,128.048919,18.311124,108.540000,108.509083,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
1,2016-02-02,91.34,91.360,91.3400,91.34,1627284.0,bil,1,-0.003626,91.386806,91.327194,46.986934,-124.094203,12.399460,91.360333,91.364333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
2,2016-02-02,372.93,374.410,371.1700,371.33,6817.0,btcusd,1,-12.397434,422.125588,352.168412,45.804685,-84.272928,40.704445,405.695000,419.803333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
3,2016-02-02,108.09,108.180,107.3500,107.92,6656018.0,gld,1,1.135824,108.620100,102.477820,56.790972,143.671556,35.269843,104.479307,103.650320,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
4,2016-02-02,190.16,191.970,189.5400,191.96,182564890.0,spy,1,-2.744645,199.106331,183.296129,43.397377,-51.889822,20.952982,195.478153,201.393583,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
5,2016-02-02,99.93,101.300,99.5600,101.23,806243.0,vb,1,-2.286971,107.259181,95.234819,39.656457,-58.511357,32.436105,104.496667,109.190167,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
6,2016-02-02,76.42,77.000,75.9900,76.75,6405965.0,vnq,1,-0.590040,80.000387,73.562613,46.993766,-52.042083,19.794014,77.738333,78.155167,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
7,2016-02-02,108.76,110.390,108.3705,110.39,1044394.0,vo,1,-2.324530,116.708953,104.273047,39.842589,-62.826827,28.141177,113.705333,118.021333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
8,2016-02-02,29.61,30.074,29.5250,30.01,29716675.0,vwo,1,-0.602581,31.511900,28.106100,40.930317,-50.822375,20.542572,30.827333,32.391667,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
9,2016-02-03,109.31,109.640,109.2000,109.26,3700282.0,agg,2,0.219987,109.409405,108.289595,56.284904,134.384090,26.433157,108.571000,108.518083,"[[5.6805382532148396e-06, 1.0046908129070857e-...",tic agg bil btcusd ...


## Training data split: 2009-01-01 to 2020-07-01

In [23]:
trade = data_split(df, TRADE_START_DATE, TRADE_END_DATE)

In [24]:
%pip install PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


In [25]:
from pypfopt.efficient_frontier import EfficientFrontier

In [26]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [27]:
df.head()

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2016-02-02,109.32,109.38,109.1301,109.23,4835115.0,agg,1,0.207871,109.414436,108.155564,56.401620,128.048919,18.311124,108.540000,108.509083,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
1,2016-02-02,91.34,91.36,91.3400,91.34,1627284.0,bil,1,-0.003626,91.386806,91.327194,46.986934,-124.094203,12.399460,91.360333,91.364333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
2,2016-02-02,372.93,374.41,371.1700,371.33,6817.0,btcusd,1,-12.397434,422.125588,352.168412,45.804685,-84.272928,40.704445,405.695000,419.803333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
3,2016-02-02,108.09,108.18,107.3500,107.92,6656018.0,gld,1,1.135824,108.620100,102.477820,56.790972,143.671556,35.269843,104.479307,103.650320,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
4,2016-02-02,190.16,191.97,189.5400,191.96,182564890.0,spy,1,-2.744645,199.106331,183.296129,43.397377,-51.889822,20.952982,195.478153,201.393583,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...


In [28]:
#calculate_portfolio_minimum_variance
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital

for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    #portfolio allocation
    ef_min_var = EfficientFrontier(None, Sigma,weight_bounds=(0, 0.15))
    #minimum variance
    raw_weights_min_var = ef_min_var.min_volatility()
    #get weights
    cleaned_weights_min_var = ef_min_var.clean_weights()

    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]
    # current held shares
    current_shares = list(np.array(current_cash)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

portfolio=portfolio.T
portfolio.columns = ['account_value']

In [29]:
portfolio.head()

,account_value
2023-04-04,1000000
2023-04-05,997298.400512
2023-04-06,998704.462984
2023-04-10,997813.397911
2023-04-11,1001406.118015


In [37]:
# Assuming 'portfolio' is your DataFrame with datetime index and 'account_value'
df_daily_return = portfolio.copy()

# Calculate percent change
df_daily_return["daily_return"] = df_daily_return["account_value"].pct_change()

# Force proper dtype inference to avoid future warning
df_daily_return = df_daily_return.infer_objects(copy=False)

# Reset index and rename columns
df_daily_return = df_daily_return.reset_index().rename(columns={"index": "date"})

# Set first daily return to 0.0
df_daily_return.loc[0, "daily_return"] = 0.0

# Keep only required columns
df_daily_return = df_daily_return[["date", "daily_return"]]



/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/ipykernel_25925/531201527.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_daily_return["daily_return"] = df_daily_return["account_value"].pct_change()


In [38]:
# Preview result
df_daily_return.head()

,date,daily_return
0,2023-04-04,0.000000
1,2023-04-05,-0.002702
2,2023-04-06,0.001410
3,2023-04-10,-0.000892
4,2023-04-11,0.003601


In [39]:
df_daily_return.to_csv('df_daily_return_min.csv')